In [5]:
import smtplib
from email.message import EmailMessage
import requests

def foundP(x):
    res=requests.get('https://ipinfo.io/')
    data=res.json()
    pcity=data['city']
    pregion=data['region']
    pcountry=data['country']
    ploc=data['loc'].split(',')
    plattitude=ploc[0]
    plongitude=ploc[1]
    content=x + " IS FOUND AT  \n" + "city :" + pcity +"\n" + "region :" + pregion + "\n" + "country :" + pcountry +"\n" + "location : " + plattitude + "," + plongitude
    content=str(content)
    msg = EmailMessage()
    msg.set_content(content)
    msg['subject']="one missing person found."
    msg['to']="rishabhraj1310@gmail.com"

    user="alert.email.missing.person@gmail.com"
    msg['from']=user
    password="akmkrkndbizkmtfd"

    server=smtplib.SMTP("smtp.gmail.com",587)
    server.starttls()
    server.login(user,password)
    server.send_message(msg)
    server.quit()

In [8]:
count=0
import numpy as np
from keras.models import model_from_json
import operator
import cv2
import sys, os
from skimage.transform import resize

# Loading the model
json_file = open("model-bw.json", "r")
model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(model_json)
# load weights into new model
loaded_model.load_weights("model-bw.h5")
print("Loaded model from disk")
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)

# Category dictionary
#categories = {0: 'ZERO', 1: 'ONE', 2: 'TWO', 3: 'THREE', 4: 'FOUR', 5: 'FIVE'}

while True:
    _, frame = cap.read()
    # Simulating mirror image
    frame = cv2.flip(frame, 1)
    
    # Got this from collect-data.py
    # Coordinates of the ROI
    x1 = int(0.5*frame.shape[1])
    y1 = 10
    x2 = frame.shape[1]-10
    y2 = int(0.5*frame.shape[1])
    # Drawing the ROI
    # The increment/decrement by 1 is to compensate for the bounding box
    #cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255,0,0) ,1)
    # Extracting the ROI
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    roi = frame[y1:y2, x1:x2]
    
    # Resizing the ROI so it can be fed to the model for prediction
    test_image = cv2.resize(roi, (64, 64)) 
    #roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    #_, test_image = cv2.threshold(roi, 120, 255, cv2.THRESH_BINARY)
    cv2.imshow("test", test_image)
    # Batch of 1
    test_image= resize(frame,(64,64))
    test_image=np.expand_dims(test_image,axis=0)
    if(np.max(test_image)>1):
        test_image= test_image/255.0
    result = loaded_model.predict(test_image)
    prediction = {'Isha': result[0][0], 
                  'Rishabh': result[0][1], 
                  'Riya': result[0][2]
                  }
    # Sorting based on top prediction
    prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
    # Displaying the predictions
    cv2.putText(frame, prediction[0][0], (10, 120), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)    
    cv2.imshow("Frame", frame)
    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == 27: # esc key
        break
        
 
cap.release()
cv2.destroyAllWindows()

if(prediction[0][0]=="Rishabh"):
    foundP(prediction[0][0])
print(prediction[0][0])







Loaded model from disk
Rishabh
